# <h2 ><div style = "background-color: #6600cc; color:white; border-radius: 30px 5px; padding: 20px; margin: 2px;"><b><span>Wide & Deep Recommender</span></b></div></h2>

In [1]:
import tensorflow 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, AveragePooling1D,Lambda,Reshape,concatenate
from tensorflow.keras import Model
from keras import backend as K
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from tensorflow.keras.layers import Embedding
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


In [2]:

COLUMNS = ["age", "workclass", "fnlwgt", "education", "education_num",
           "marital_status", "occupation", "relationship", "race", "gender",
           "capital_gain", "capital_loss", "hours_per_week", "native_country",
           "income_bracket"]


def load_df(filename):
	with open(filename, 'r') as f:
		df = pd.read_csv(f, names = COLUMNS, skipinitialspace=True, engine='python')
	return df



In [3]:
df_train=load_df('../input/adult-data/adult.data')
df_train_deep=load_df('../input/adult-data/adult.data')
df_test=load_df('../input/adult-data/adult.test')
df_test_deep=load_df('../input/adult-data/adult.test')
df_test=df_test.drop(index=0)
df_test_deep=df_test_deep.drop(index=0)

In [4]:

# infome_label추가
df_train['income_label'] = (df_train["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
df_train_deep['income_label'] = (df_train_deep["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
#df_test['income_label'] = (df_test["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)

# age_group 추가
age_groups = [0, 25, 65, 90]
age_labels = range(len(age_groups) - 1)
df_train['age_group'] = pd.cut(df_train['age'], age_groups, labels=age_labels)
df_train_deep['age_group'] = pd.cut(df_train_deep['age'], age_groups, labels=age_labels)
target = 'income_label'


#age group test
df_test['age_group'] = pd.cut(df_test['age'].astype(int), age_groups, labels=age_labels)
df_test['income_label'] = (df_test["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
df_test_deep=load_df('../input/adult-data/adult.test')
df_test_deep['income_label'] = (df_train_deep["income_bracket"].apply(lambda x: ">50K" in x)).astype(int)
df_test_deep['age_group'] = pd.cut(df_train_deep['age'], age_groups, labels=age_labels)

In [5]:
df_train['IS_TRAIN'] = 1
df_test['IS_TRAIN'] = 0
df_wide = pd.concat([df_train, df_test])

In [6]:
 def cross_columns(x_cols):

        crossed_columns = dict()
        colnames = ['_'.join(x_c) for x_c in x_cols]
        for cname, x_c in zip(colnames, x_cols):
            crossed_columns[cname] = x_c
        return crossed_columns

In [7]:
wide_cols = ['workclass', 'education', 'marital_status', 'occupation','relationship', 'race', 'gender', 'native_country', 'age_group']
crossed_cols = (['education', 'occupation'], ['native_country', 'occupation'])
target = 'income_label'

In [8]:
categorical_columns = list(df_train.select_dtypes(include=['object']).columns)
crossed_columns_dic = cross_columns(crossed_cols)
wide_cols += list(crossed_columns_dic.keys())

In [9]:
wide_cols

In [10]:
for col_name, col_lst in crossed_columns_dic.items():
            print(col_name,col_lst)
            df_wide[col_name] = df_wide[col_lst].apply(lambda x: '-'.join(x), axis=1)

In [11]:
df_wide = df_wide[wide_cols + [target]+['IS_TRAIN']]
#onehot encoding
dummy_cols = [wc for wc in wide_cols if wc in categorical_columns + list(crossed_columns_dic.keys())]
df_wide = pd.get_dummies(df_wide, columns=dummy_cols)

In [12]:
train_wide = df_wide[df_wide.IS_TRAIN == 1].drop('IS_TRAIN', axis=1)
test_wide = df_wide[df_wide.IS_TRAIN == 0].drop('IS_TRAIN', axis=1)

In [13]:
#deep data#categorical data

embedding_cols = ['workclass', 'education', 'marital_status', 'occupation',
                          'relationship', 'race', 'gender', 'native_country']
        
cont_cols = ['age', 'capital_gain', 'capital_loss', 'hours_per_week']

target = ['income_label']
deep_cols = embedding_cols + cont_cols+target+['IS_TRAIN']



In [14]:
 df_deep = pd.concat([df_train, df_test])

In [15]:
df_deep=df_deep[deep_cols]

In [16]:
le = preprocessing.LabelEncoder()
for cat_col in embedding_cols:
    #le.fit(df_train_deep[cat_col])
    df_deep[cat_col]=le.fit_transform(df_deep[cat_col])
#    df_test_deep[cat_col]=le.fit_transform(df_test_deep[cat_col])
    
    



In [17]:
scaler = StandardScaler()
df_deep[cont_cols] = pd.DataFrame(scaler.fit_transform(df_deep[cont_cols]), columns=cont_cols)


In [18]:
train_deep = df_deep[df_deep.IS_TRAIN == 1].drop('IS_TRAIN', axis=1)
test_deep = df_deep[df_deep.IS_TRAIN == 0].drop('IS_TRAIN', axis=1)

In [19]:
#embedding
embeddings_tensors=[]
for ec in embedding_cols:
            un=df_train_deep[ec].nunique()
            
            layer_name = ec + '_inp'
            inp = Input(shape=(1,), dtype='int64', name=layer_name)
            embd = Embedding(un, 8, input_length=1, embeddings_regularizer=l2(1e-3))(inp)
            embeddings_tensors.append((inp, embd))
            del (inp, embd)


In [20]:
continuous_tensors = []
for cc in cont_cols:
            layer_name = cc + '_in'
            inp = Input(shape=(1,), dtype='float32', name=layer_name)
            bulid = Reshape((1, 1))(inp)
            continuous_tensors.append((inp, bulid))
            del (inp, bulid)

In [21]:
x_train_deep_c=[c for c in deep_cols if c not in  target +['IS_TRAIN']]
x_train_deep=[train_deep[c] for c in x_train_deep_c]
x_test_deep=[test_deep[c] for c in x_train_deep_c]

y_train_deep = np.array(train_deep[target].values).reshape(-1, 1)

In [22]:
x_train_wide_c=[c for c in train_wide.columns  if c not in  target +['IS_TRAIN']]
x_train_wide=train_wide[x_train_wide_c].values
x_train_wide = np.array(x_train_wide, dtype=np.float)

x_test_wide=test_wide[x_train_wide_c].values
x_test_wide = np.array(x_test_wide, dtype=np.float)


y_train_wide = np.array(train_wide[target].values).reshape(-1, 1)



In [23]:
#lets create the archetecture
input_wide=Input(shape=(x_train_wide.shape[1],))
deep_input = [et[0] for et in embeddings_tensors] + [ct[0] for ct in continuous_tensors]
deep_embedding = [et[1] for et in embeddings_tensors] + [ct[1] for ct in continuous_tensors]
deep_embedding = Flatten()(concatenate(deep_embedding))
logi_d1=Dense(128,activation='relu')(deep_embedding)
logi_d2=Dense(56,activation='relu')(logi_d1)
logi_d3=Dense(28,activation='relu')(logi_d2)

wd_conct=concatenate([input_wide,logi_d3])
wd_log=Dense(1,activation='relu')(wd_conct)

model = Model([input_wide,deep_input], wd_log)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
x_tr_wd =[x_train_wide]+x_train_deep

In [25]:
x_ts_wd =[x_test_wide]+x_test_deep

In [26]:
model.fit(x_tr_wd, y_train_wide, epochs=18, batch_size=128)

In [27]:
predicted=model.predict(x_ts_wd)

y_test = np.array(test_deep[target].values).reshape(-1, 1)

In [28]:
predicted[predicted> .5] = 1
predicted[predicted<= .5] = 0

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted))

In [37]:
from sklearn.metrics import auc,roc_curve

In [42]:
fpr, tpr, thresholds = roc_curve(y_test, predicted)

auc(fpr, tpr)

In [30]:
model.summary()

In [31]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)